In [1]:
from azureml.core import Workspace, Experiment, Model

# ws = Workspace.get(name="capstone")
ws = Workspace.from_config()

# exp = Experiment(workspace=ws, name="udacity-project-hyperparam-bayes")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: capstone
Azure region: westeurope
Subscription id: 72f46e0e-1451-4b79-92cd-fc8f7797bda7
Resource group: test-Jesse


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# from https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/train-on-amlcompute/train-on-amlcompute.ipynb
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [2]:
explanation = { 'udacity-project-hyperparam-bandit' : 'hyperdrive xgboost with bandit policy',
                'udacity-project-hyperparam-bayes' : 'hyperdrive xgboost with bayesian sampling',
                'automl': 'AutoML on same dataset as hyperdrive',
                'AutoML-onlylogtransform': 'AutoML on original dataset (except log transform target)',
                'automl-kaggle-orgininal': 'AutoML on original dataset (no log transform target)'
}

preprocessed_experiments = ['udacity-project-hyperparam-bandit', 'udacity-project-hyperparam-bayes','automl']
only_log_experiment = ['AutoML-onlylogtransform']
original_experiment = ['automl-kaggle-orgininal']

all_exp = preprocessed_experiments + only_log_experiment + original_experiment

def get_score(run):

    # return t.get_metrics()['rmse'] if run.status == 'Completed' else None
    print(float(run.get_metrics()['rmse']) if run.status == 'Completed' else None)
    return float(run.get_metrics()['rmse']) if run.status == 'Completed' else None


def get_bestrun(exp_name):
    print(explanation[exp_name], end='\t')
    exp = Experiment(workspace=ws, name=exp_name)
    run = list(exp.get_runs())[0]
    if run.type == 'hyperdrive':
        best_run = run.get_best_run_by_primary_metric()
        print(round(best_run.get_metrics()['rmse'],3))
    elif run.type == 'automl':
        best_run = run.get_best_child()
        print(round(best_run.get_metrics()['root_mean_squared_error'],3))
    else:
        print('wrong type')
    return best_run

    
    

In [7]:
exp = Experiment(workspace=ws, name='automl')
run = list(exp.get_runs())[0]


In [14]:
get_bestrun('automl')

AutoML on same dataset as hyperdrive	0.117


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_6447f742-e7cc-4620-be3f-086c14a7ef61_43,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
best_run, best_model = run.get_output()
model = best_run.register_model(model_name = 'automl', model_path = './outputs/')
# joblib.dump(model, './outputs/best_automl_model.joblib')
# run.get_output()

In [12]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_6447f742-e7cc-4620-be3f-086c14a7ef61_43,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
bestruns = [get_bestrun(run_name) for run_name in all_exp]

hyperdrive xgboost with bandit policy	0.126
hyperdrive xgboost with bayesian sampling	0.127
AutoML on same dataset as hyperdrive	0.117
AutoML on original dataset (except log transform target)	0.125
AutoML on original dataset (no log transform target)	26738.501


In [15]:
bestruns[2].register_model(
   model_name='automl2',
   model_path="./outputs/")

Model(workspace=Workspace.create(name='capstone', subscription_id='72f46e0e-1451-4b79-92cd-fc8f7797bda7', resource_group='test-Jesse'), name=automl2, id=automl2:2, version=2, tags={}, properties={})

In [29]:
import joblib
model = joblib.load('outputs/model.pkl')
model.predict()

TypeError: predict() missing 1 required positional argument: 'X'

In [24]:
Model.

ModelNotFoundException: ModelNotFoundException:
	Message: Model /outputs/model.pkl not found in cache at azureml-models or in current working directory d:\Documenten\GitHub\courses\Nanodegree Azure Machine Learning\Azure ML Capstone. For more info, set logging level to DEBUG.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Model /outputs/model.pkl not found in cache at azureml-models or in current working directory d:\\Documenten\\GitHub\\courses\\Nanodegree Azure Machine Learning\\Azure ML Capstone. For more info, set logging level to DEBUG."
    }
}

In [7]:
exp_name = all_exp[2]
print(explanation[exp_name], end='\t')
exp = Experiment(workspace=ws, name=exp_name)
run = list(exp.get_runs())[0]
if run.type == 'hyperdrive':
    best_run = run.get_best_run_by_primary_metric()
    print(round(best_run.get_metrics()['rmse'],3))
elif run.type == 'automl':
    best_run = run.get_best_child()
    print(round(best_run.get_metrics()['root_mean_squared_error'],3))
# best_run.register_model(
#    model_name=exp_name, 
#    model_path="outputs/model.pkl")

AutoML on same dataset as hyperdrive	

AttributeError: 'Run' object has no attribute 'get_best_child'

In [13]:
exp = Experiment(workspace=ws, name='automl')
run = list(exp.get_runs())[0]
type(run)

azureml.core.run.Run

In [ ]:
from azureml.pipeline.core import PipelineRun

run_id = "b05118c5-ab36-418c-a774-ee0058d772ad"
pipeline_run = PipelineRun(experiment, run_id)

In [30]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform, quniform
import os

# Specify parameter sampler
# Did some reseach to find acceptable values. Chose loguniform to make the sampling work better
# Especially for the C parameter this is important, otherwise Azure would be unlikely to explore the very low values
# https://www.kaggle.com/joparga3/2-tuning-parameters-for-logistic-regression#Testing-different-parameters-to-understand-how-accuracies-change.
# https://stackoverflow.com/questions/21816346/fine-tuning-parameters-in-logistic-regression
ps = BayesianParameterSampling( {
    'max_depth': quniform(2, 10,1), #9,12
    'min_child_weight': quniform(0, 4,1), # this is a Real instead of integer # if leaves with small amount of observations are allowed?
    # 'min_child_weight': Real(0, 4), # this is a Real instead of integer # if leaves with small amount of observations are allowed?
    'gamma' : uniform(0,0.15), # these 3 for model complexity. gemma is a threshold for gain of the new split. 
    'subsample': uniform(0.6,1.0), # (0.5,1)
    # 'subsample': Real(0.5,1.0), 
    'colsample_bytree' : uniform(0.5,1.0), # these 3 for making model more robust to noise
    'reg_lambda' : uniform(0.0,0.9), # regularization lambda, reduces similarity scores and therefore lowers gain. reduces sensitivity to individual observations
    # 'reg_alpha' : Real(0.3,0.8), # regularization lambda, reduces similarity scores and therefore lowers gain. reduces sensitivity to individual observations
    'colsample_bylevel': uniform(0.60,1.0),
    'learning_rate': uniform(0.001,0.4), # 0.3 is default
    'max_delta_step': uniform(0.01,10.0),
    'n_estimators': quniform(10,4000,1)})


# Specify a Policy
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# SKIPPED BECAUSE ESTIMATORS ARE DEPRECIATED 
# https://knowledge.udacity.com/questions/407297
# Essentially I started following https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
# this means I have to make a scriptrunconfig first and submit the hyperdriveconfig

In [31]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.core import Environment, ScriptRunConfig

project_folder = '.'
myenv = Environment.from_conda_specification(name = 'project1', file_path = '.azureml/project1.yml')
# myenv = Environment("user-managed-env")
# myenv.python.user_managed_dependencies = True
src = ScriptRunConfig(source_directory=project_folder, 
                      script='train.py', 
                      compute_target=cpu_cluster, 
                    #   compute_target='local', 
                      environment=myenv)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=None,
                             primary_metric_name="rmse",
                             primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                             max_total_runs=220,
                             max_concurrent_runs=4)

hyperdrive_run = exp.submit(config=hyperdrive_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-hyperparam-bayes,HD_31c58aff-6eca-464e-82dc-9228dee03f8c,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
exp = Experiment(workspace=ws, name="capstone-hyperparam")
hyperdrive_run = list(exp.get_runs())[0]

In [32]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [31]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run_metrics['Accuracy'])
files = best_run.get_file_names()
best_run.download_file(files[-1], output_file_path='./outputs/')
joblib.load('./outputs/model.joblib')

['--C', '3.145839853194323E+279', '--max_iter', '100']
0.9088012139605463


LogisticRegression(C=3.145839853194323e+279, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
# clean up compute cluster
cpu_cluster.delete()

# AutoML part

In [18]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="capstone")

In [19]:
def submit(model, filename='submission.csv'):
    pred = model.predict(final_test)
    final_test['SalePrice'] = np.exp(pred)
    final_test[['Id','SalePrice']].to_csv('output/'+filename, index=False)

In [20]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
train_url = ["https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/housing_after_preprocessing.csv"]
train = TabularDatasetFactory.from_delimited_files(train_url)
test_url = ["https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/final_test.csv"]
test = TabularDatasetFactory.from_delimited_files(test_url)

In [21]:
df = train.take(5).to_pandas_dataframe()
df.head()

,Column1,Total_sqr_footage,GarageFinish,OverallCond,AllPorchSF,Condition1,OverallQual_3,AllSF,Neighborhood,SimplOverallCond,...,ScreenPorch,GrLivArea_log,GarageGrade,Fireplaces_log,BsmtFullBath_log,ExterQual_2,SimplOverallQual_3,GarageCars_2,SimplKitchenQual,y
0,0,2416,RFn,5,61,Norm,343,2566,CollgCr,2,...,0,7.444833,9.0,0.000000,0.693147,16,27,4,2,12.247699
1,1,2240,RFn,8,298,Feedr,216,2524,Veenker,3,...,0,7.141245,9.0,0.693147,0.000000,9,8,4,1,12.109016
2,2,2272,RFn,5,42,Norm,343,2706,CollgCr,2,...,0,7.488294,9.0,0.693147,0.693147,16,27,4,2,12.317171
3,3,1933,Unf,5,307,Norm,343,2473,Crawfor,2,...,0,7.448916,9.0,0.693147,0.693147,9,27,9,2,11.849405
4,4,2853,RFn,5,276,Norm,512,3343,NoRidge,2,...,0,7.695758,9.0,0.693147,0.693147,16,27,9,2,12.429220


In [12]:
train = train.register(workspace=ws,
                                 name='train',
                                 description='training data')

test = test.register(workspace=ws,
                                 name='test',
                                 description='test data for kaggle submission')

In [13]:
# I have to convert the dataframe to a dataset since I'm running jupyter in VSCode locally
from azureml.core import Dataset
def upload(ws, df, name):
    local_path = 'data/'+name+'.csv'
    df.to_csv(local_path)
    datastore = ws.get_default_datastore()
    # upload the local file from src_dir to the target_path in datastore
    datastore.upload(src_dir='data', target_path='data')
    # create a dataset referencing the cloud location
    return Dataset.Tabular.from_delimited_files(path = [(datastore, (local_path))])

train_ds = upload(ws, train, 'train')
test_ds = upload(ws, train, 'test')

AttributeError: 'TabularDataset' object has no attribute 'to_csv'

In [22]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-2"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# changed validation set into n_cross_validation for review

automl_config = AutoMLConfig(
    experiment_timeout_minutes=60,
    enable_onnx_compatible_models=True,
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    training_data= train,
    compute_target=cpu_cluster,
    label_column_name='y',
    n_cross_validations=5)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [23]:
# Submit your automl run
from azureml.core import Workspace, Experiment
experiment=Experiment(ws, "automl")
automl_run = experiment.submit(automl_config, show_output = False)

Running on remote.


In [24]:
automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_6447f742-e7cc-4620-be3f-086c14a7ef61,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [31]:
import joblib
best_run, fitted_model = automl_run.get_output()
joblib.dump(fitted_model, './outputs/best_automl_model.joblib')
joblib.load('./outputs/best_automl_model.joblib')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.01,
                                                                                                    min_weight_fraction_leaf=0.0,
               

In [32]:
# clean up compute cluster
cpu_cluster.delete()